<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/resumeTrainingFromLastState_save_and_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


### Get an example dataset

To demonstrate how to save and load weights, you'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). To speed up these runs, use the first 1000 examples:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Start by building a simple sequential model:

In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The `tf.keras.callbacks.ModelCheckpoint` callback allows you to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [ ]:
checkpoint_path = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

Epoch 1/10
32/32 [==============================] - 1s 21ms/step - loss: 1.5810 - sparse_categorical_accuracy: 0.4919 - val_loss: 0.7508 - val_sparse_categorical_accuracy: 0.7750

Epoch 00001: saving model to training_2/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 0.4654 - sparse_categorical_accuracy: 0.8779 - val_loss: 0.5412 - val_sparse_categorical_accuracy: 0.8320

Epoch 00002: saving model to training_2/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 6ms/step - loss: 0.3153 - sparse_categorical_accuracy: 0.9293 - val_loss: 0.4669 - val_sparse_categorical_accuracy: 0.8560

Epoch 00003: saving model to training_2/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 0.2225 - sparse_categorical_accuracy: 0.9493 - val_loss: 0.4374 - val_sparse_categorical_accuracy: 0.8620

Epoch 00004: saving model to training_2/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 6ms/step - loss: 0.1853 - sparse_cat

In [ ]:
# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

Epoch 1/10
32/32 [==============================] - 0s 7ms/step - loss: 0.0285 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4018 - val_sparse_categorical_accuracy: 0.8710

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0282 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.4076 - val_sparse_categorical_accuracy: 0.8720

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0206 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4239 - val_sparse_categorical_accuracy: 0.8740

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0155 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4155 - val_sparse_categorical_accuracy: 0.8770

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0154 - sparse_cate

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [ ]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights. 

Now rebuild a fresh, untrained model and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [ ]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3406 - sparse_categorical_accuracy: 0.1040
Untrained model, accuracy: 10.40%


Then load the weights from the checkpoint and re-evaluate:

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4795 - sparse_categorical_accuracy: 0.8800
Restored model, accuracy: 88.00%


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


Now, look at the resulting checkpoints and choose the latest one:

In [ ]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

Note: the default TensorFlow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5011 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


In [ ]:
import numpy as np
from numpy.testing import assert_allclose
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

vec_size = 100
n_units = 10

x_train = np.random.rand(500, 10, vec_size)
y_train = np.random.rand(500, vec_size)

model = Sequential()
model.add(LSTM(n_units, input_shape=(None, vec_size), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(n_units, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(n_units))
model.add(Dropout(0.2))
model.add(Dense(vec_size, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

# define the checkpoint
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(x_train, y_train, epochs=5, batch_size=50, callbacks=callbacks_list)

# load the model
new_model = load_model("model.h5")
assert_allclose(model.predict(x_train),
                new_model.predict(x_train),
                1e-5)

# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
new_model.fit(x_train, y_train, epochs=5, batch_size=50, callbacks=callbacks_list)

Epoch 1/5
10/10 [==============================] - 31s 5ms/step - loss: 0.3237

Epoch 00001: loss improved from inf to 0.31665, saving model to model.h5
Epoch 2/5
10/10 [==============================] - 0s 5ms/step - loss: 0.2894

Epoch 00002: loss improved from 0.31665 to 0.28459, saving model to model.h5
Epoch 3/5
10/10 [==============================] - 0s 5ms/step - loss: 0.2534

Epoch 00003: loss improved from 0.28459 to 0.24435, saving model to model.h5
Epoch 4/5
10/10 [==============================] - 0s 5ms/step - loss: 0.2084

Epoch 00004: loss improved from 0.24435 to 0.19932, saving model to model.h5
Epoch 5/5
10/10 [==============================] - 0s 5ms/step - loss: 0.1679

Epoch 00005: loss improved from 0.19932 to 0.16083, saving model to model.h5
Epoch 1/5
10/10 [==============================] - 1s 5ms/step - loss: 0.1346

Epoch 00001: loss improved from inf to 0.13465, saving model to model.h5
Epoch 2/5
10/10 [==============================] - 0s 5ms/step - loss: 